<a href="https://colab.research.google.com/github/EricEsajian/forecasting-models/blob/master/3dKnapsack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install py3dbp
!pip install ortools
import numpy as np
import pandas as pd
from py3dbp import Packer, Bin, Item
from tqdm import tqdm
from pandas_profiling import ProfileReport
from ortools.algorithms import pywrapknapsack_solver

In [ ]:
order_sku_df = pd.read_excel('/content/sample_data/Order Info.xlsx')
sku_dimenssion_df = pd.read_excel('/content/sample_data/Sku.xlsx')

In [ ]:
df = order_sku_df.copy()
df = df.merge(sku_dimenssion_df, how='left', on='SKU_ID')
df['Volume']=np.round(df['Volume']*1000,0).astype('int')
df.sort_values('Order_No',inplace=True)
df.head(5)

,Order_No,SKU_ID,Quantity,Priority_Value,Length,Width,Height,Volume
149,1,190,50,66,1.0,0.4,0.3,120
453,1,73,29,58,1.0,0.3,0.6,180
24,1,126,17,86,1.3,0.9,0.4,468
43,1,414,22,13,0.6,0.3,0.6,108
81,1,198,47,70,0.3,0.4,0.5,60


In [ ]:
np.sum((df['Volume']*df['Quantity']))


3029269

In [ ]:
df['Total_Volume']=df['Volume']*df['Quantity']
grouped=df.drop(columns=['Length','Width','Height','SKU_ID','Volume','Quantity'],axis=0).groupby('Order_No').sum()
grouped

,Priority_Value,Total_Volume
Order_No,,
1,382,27804
2,487,47794
3,207,30268
4,229,23490
5,425,49442
...,...,...
96,333,39315
97,307,29981
98,82,12264


In [ ]:
from ortools.algorithms import pywrapknapsack_solver

def solved(values,weights,capacities):
    # Create the solver.
    solver = pywrapknapsack_solver.KnapsackSolver(
        pywrapknapsack_solver.KnapsackSolver.
        KNAPSACK_MULTIDIMENSION_BRANCH_AND_BOUND_SOLVER, 'KnapsackExample2')
    solver.Init(values, weights, capacities)
    computed_value = solver.Solve()

    packed_items = []
    packed_weights = []
    total_weight = 0
    print('Total value =', computed_value)
    for i in range(len(values)):
        if solver.BestSolutionContains(i):
            packed_items.append(i)
            packed_weights.append(weights[0][i])
            total_weight += weights[0][i]
    print('Total weight:', total_weight)
    print('Packed items:', packed_items)
    print('Packed_weights:', packed_weights)
    return(packed_items)



In [ ]:
values = df['Priority_Value'].tolist()
weights = [(df['Total_Volume']).tolist()]
capacities = [70000]

In [ ]:
packed_items=solved(values,weights,capacities)

Total value = 5444
Total weight: 69972
Packed items: [7, 16, 19, 23, 32, 34, 35, 40, 50, 55, 61, 65, 74, 76, 78, 95, 96, 99, 101, 109, 110, 111, 120, 125, 147, 153, 154, 170, 174, 179, 182, 185, 187, 190, 197, 200, 209, 216, 218, 223, 249, 251, 255, 259, 263, 270, 272, 279, 286, 311, 315, 328, 332, 336, 347, 350, 354, 361, 363, 365, 368, 383, 386, 392, 420, 422, 432, 440, 447, 449, 451, 452, 456, 481, 482, 485, 486, 491, 496]
Packed_weights: [1600, 720, 1134, 896, 1080, 360, 1566, 1425, 936, 168, 882, 1323, 576, 1680, 420, 630, 336, 864, 480, 630, 126, 2205, 546, 2340, 1560, 750, 275, 1404, 480, 1020, 1188, 588, 1200, 480, 1512, 240, 720, 360, 896, 270, 648, 1296, 2205, 270, 504, 640, 972, 1760, 513, 1680, 1440, 540, 1152, 960, 672, 1008, 640, 1728, 315, 945, 700, 1365, 936, 1296, 270, 168, 720, 225, 720, 270, 360, 540, 1980, 750, 1080, 1395, 336, 882, 225]


In [ ]:
df2=df.iloc[packed_items]

In [ ]:
df2.head()


,Order_No,SKU_ID,Quantity,Priority_Value,Length,Width,Height,Volume,Total_Volume
146,2,67,16,97,0.4,0.5,0.5,100,1600
200,3,28,3,55,1.0,0.8,0.3,240,720
245,4,412,9,84,1.4,0.3,0.3,126,1134
400,5,240,7,41,0.4,0.8,0.4,128,896
271,5,57,40,60,0.3,0.3,0.3,27,1080


In [ ]:
df3=df2.copy()
df3.loc[:,'Order_No']=df3['Order_No'].astype('int')
df3.loc[:,'SKU_ID']=df3['SKU_ID'].astype('string')
df3.loc[:,'Quantity']=df3['Quantity'].astype('int')
df3.drop('Volume',axis=1)

,Order_No,SKU_ID,Quantity,Priority_Value,Length,Width,Height,Total_Volume
146,2,67,16,97,0.4,0.5,0.5,1600
200,3,28,3,55,1.0,0.8,0.3,720
245,4,412,9,84,1.4,0.3,0.3,1134
400,5,240,7,41,0.4,0.8,0.4,896
271,5,57,40,60,0.3,0.3,0.3,1080
...,...,...,...,...,...,...,...,...
304,97,369,9,91,0.5,0.8,0.3,1080
383,97,448,31,95,0.3,0.5,0.3,1395
217,97,295,7,23,0.4,0.4,0.3,336
325,99,97,6,56,0.7,0.7,0.3,882


In [ ]:
packerX = Packer()
my_bin = Bin('40ft container',12.025,2.280,2.352,1)
packerX.add_bin(my_bin)
for _, row in df3.iterrows():
    for _ in range(row['Quantity']):
              packerX.add_item(Item(row['SKU_ID'], row['Length'], row['Width'], row['Height'], 0))
packerX.pack()



In [ ]:
import regex as re
fitted_ordNo=[]
fitted_weights=[]
fitted_rotations=[]
fitted_positions=[]
fitted_volumes=[]
unfitted_ordNo=[]
unfitted_weights=[]
unfitted_rotations=[]
unfitted_positions=[]
unfitted_volumes=[]
for b in packerX.bins:
    for item in b.items:
        fitted_ordNo.append(item.string().split("(")[0])
    for item in b.items:
        fitted_weights.append(re.findall('\(.*?\)',item.string())[0])
    for item in b.items:
        fitted_positions.append(re.findall('\[.*?\]',item.string())[0])
    for item in b.items:
        fitted_rotations.append(re.findall('\(.*?\)',item.string())[-2][re.findall('\(.*?\)',item.string())[-2].find("(")+1:re.findall('\(.*?\)',item.string())[-2].find(")")])
    for item in b.items:
        fitted_volumes.append(re.findall('\(.*?\)',item.string())[-1][re.findall('\(.*?\)',item.string())[-1].find("(")+1:re.findall('\(.*?\)',item.string())[-1].find(")")]) 
    fitted_df=pd.DataFrame({'Order_Number':fitted_ordNo,'Weight&Dim':fitted_weights,'Position':fitted_positions, 'Rotation':fitted_rotations,'Volume':fitted_volumes})
    fitted_df.loc[:,'Volume']=fitted_df['Volume'].astype(float)
    for item in b.unfitted_items:
        unfitted_ordNo.append(item.string().split("(")[0])
    for item in b.unfitted_items:
        unfitted_weights.append(re.findall('\(.*?\)',item.string())[0])
    for item in b.unfitted_items:
        unfitted_positions.append(re.findall('\[.*?\]',item.string())[0])
    for item in b.unfitted_items:
        unfitted_rotations.append(re.findall('\(.*?\)',item.string())[-2])  
    for item in b.unfitted_items:
        unfitted_volumes.append(re.findall('\(.*?\)',item.string())[-1])  
    unfitted_df=pd.DataFrame({'Order_Number':unfitted_ordNo,'Weight&Dim':unfitted_weights,'Position':unfitted_positions, 'Rotation':unfitted_rotations,'Volume':unfitted_volumes})
    
for b in packerX.bins:
    print(":::::::::::", b.string())





::::::::::: 40ft container(12.025x2.280x2.352, max_weight:1.000) vol(64.485)


In [ ]:
fitted_df.head(5)

,Order_Number,Weight&Dim,Position,Rotation,Volume
0,57,"(0.300x0.300x0.300, weight: 0.000)","[0, 0, 0]",0,0.027
1,57,"(0.300x0.300x0.300, weight: 0.000)","[Decimal('0.300'), 0, 0]",0,0.027
2,57,"(0.300x0.300x0.300, weight: 0.000)","[Decimal('0.600'), 0, 0]",0,0.027
3,57,"(0.300x0.300x0.300, weight: 0.000)","[Decimal('0.900'), 0, 0]",0,0.027
4,57,"(0.300x0.300x0.300, weight: 0.000)","[Decimal('1.200'), 0, 0]",0,0.027


In [ ]:
#Still work to go as the total loadable volume is quite low, but the best possible out of the mix of boxes. 
#Try increasing the knapsack volume and see if giving more flexibility to step 2 in terms of possible options will increase utilisation to acceptable levels.
#Running the notebook a few times with different knapsack volumes to see roughly where I can hit max utilisation with smallest subset
#of optimal items
fitted_df['Volume'].sum()

47.51

In [ ]:
fitted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order_Number  626 non-null    object 
 1   Weight&Dim    626 non-null    object 
 2   Position      626 non-null    object 
 3   Rotation      626 non-null    object 
 4   Volume        626 non-null    float64
dtypes: float64(1), object(4)
memory usage: 24.6+ KB


In [ ]:
unfitted_df.head(5)

,Order_Number,Weight&Dim,Position,Rotation,Volume
0,337,"(0.500x0.700x0.500, weight: 0.000)","[0, 0, 0]",(5),(0.175)
1,337,"(0.500x0.700x0.500, weight: 0.000)","[0, 0, 0]",(5),(0.175)
2,337,"(0.500x0.700x0.500, weight: 0.000)","[0, 0, 0]",(5),(0.175)
3,60,"(0.900x0.600x0.400, weight: 0.000)","[0, 0, 0]",(5),(0.216)
4,60,"(0.900x0.600x0.400, weight: 0.000)","[0, 0, 0]",(5),(0.216)


In [ ]:
unfitted_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order_Number  69 non-null     object
 1   Weight&Dim    69 non-null     object
 2   Position      69 non-null     object
 3   Rotation      69 non-null     object
 4   Volume        69 non-null     object
dtypes: object(5)
memory usage: 2.8+ KB
